**xgboost library 추가 설치시 발생된 이슈**

1. 가상 환경 구축해서 python 버전 3.6 or 3.7

2. python -m pip install xgboost(최지수 작품)
    - anaconda 콘다 명령어로 install 시도
    - 가끔은 모든 library들이 conda 명령어로 해결되지 않음
    - pip 명령어로 설치 

In [1]:
#!python -m pip install xgboost 


<hr>

1. tree 기반의 앙상블 학습에서 각광 받는 알고리즘
2. GBM 기반(GBM - 속도 이슈)
3. 속도 향상을 한 알고리즘, 과적합 방지를 GBM 보다 더 효과적
4. 뛰어난 예측 성능
5. 예측 성능이 좀 떨어지는 시점에 가지치기 즉 실행 종료(조기중단 : early stopping) / 오류기능(함수)
6. 교차검증, 성능평가, 피처 중요도 시가고하 기능을 보유

<hr>
1. 사용데이터 : 양성종양(benign, 1)?/악성종양(malignant, 0)? 데이터값 보유한 위스콘신 유방암 데이터

<hr>
XGBoost 조기중단 수행


1. fit() 함수에 조기중단 파라미터 입력
2. 조기중단 파라미터 기능 - 평가 지표 향상 될 수 있도록 반복횟수를 정의하는 early_stopping_rounds, 
조기중단을 위한 평 지표인 eval_metric(logloss 이진 분류시 예측, 정답과의 오차값 계산), eval_set은 성능평가를 수행할 데이터 셋 XGBoost 조기중단 수행
3. eval_set 으로 반복 수행시마다  지정된 데이터 세트에서 eval_metric으로 지정된 평가 지표 함수호 예측 오류 측정 

<hr>
1. 예측값이 0 또는 1로만 확실하게 예측 및 확인 
2. 참고 : c로 구성된 XGBoost도 존재, c python에서 사용 가능하게 변환해준 API 까지 다운받은 상태이기도 함
    - 때에 따라 예측시 확률로 결과를 제시(0과 1로 결과가 예측되는게 아닌 0~1 사이 데이터로 예측) <br>
    - 해결책 : 변환을 명시적으로 해야 하나 잘 구성된 API들은 0.5 기준으로 나눔(0.5보다 크면 1/ 0.5 이하면 0) <br>
3. 실습한 API : fit()/predict() 사용 가능하다는건 동일한 클래스 알고리즘 상속받아서 구현된 API들
    


**참고**

1. 데이터셋이 적은 것으로 모델링은 절대 금지
2. 데이터의 최소 적성선개수 10000 라고 하기도 함 

In [2]:
from xgboost import XGBClassifier
from xgboost import plot_importance

import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


In [3]:
dataset = load_breast_cancer()
X_features = dataset.data
y_label = dataset.target

In [4]:
y_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [5]:
X_features

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [6]:
X_features.shape, y_label.shape

((569, 30), (569,))

In [7]:
# DataFrame으로 구성
cancer_df = pd.DataFrame(data=X_features, columns=dataset.feature_names)
cancer_df['target'] = y_label
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [8]:
# 악성 : 0 / 양성 : 1
print(dataset.target_names)
print(cancer_df['target'].value_counts())

['malignant' 'benign']
1    357
0    212
Name: target, dtype: int64


In [9]:
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156 )
print(X_train.shape , X_test.shape)

(455, 30) (114, 30)


In [10]:
xbg = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)

evals = [(X_test, y_test)]

xbg.fit(X_train, y_train, early_stopping_rounds=400, eval_set=evals, eval_metric='logloss')
x_preds= xbg.predict(X_test)

[0]	validation_0-logloss:0.61352
[1]	validation_0-logloss:0.54784
[2]	validation_0-logloss:0.49425
[3]	validation_0-logloss:0.44799
[4]	validation_0-logloss:0.40911
[5]	validation_0-logloss:0.37498
[6]	validation_0-logloss:0.34571
[7]	validation_0-logloss:0.32053
[8]	validation_0-logloss:0.29721
[9]	validation_0-logloss:0.27799
[10]	validation_0-logloss:0.26030
[11]	validation_0-logloss:0.24604
[12]	validation_0-logloss:0.23156
[13]	validation_0-logloss:0.22005
[14]	validation_0-logloss:0.20857
[15]	validation_0-logloss:0.19999
[16]	validation_0-logloss:0.19012
[17]	validation_0-logloss:0.18182
[18]	validation_0-logloss:0.17473
[19]	validation_0-logloss:0.16766
[20]	validation_0-logloss:0.15820
[21]	validation_0-logloss:0.15473
[22]	validation_0-logloss:0.14895
[23]	validation_0-logloss:0.14331
[24]	validation_0-logloss:0.13634
[25]	validation_0-logloss:0.13278
[26]	validation_0-logloss:0.12791
[27]	validation_0-logloss:0.12526
[28]	validation_0-logloss:0.11998
[29]	validation_0-loglos

In [11]:
xbg = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)

evals = [(X_test, y_test)]

xbg.fit(X_train, y_train, early_stopping_rounds=100, eval_set=evals, eval_metric='logloss')
x_preds= xbg.predict(X_test)

[0]	validation_0-logloss:0.61352
[1]	validation_0-logloss:0.54784
[2]	validation_0-logloss:0.49425
[3]	validation_0-logloss:0.44799
[4]	validation_0-logloss:0.40911
[5]	validation_0-logloss:0.37498
[6]	validation_0-logloss:0.34571
[7]	validation_0-logloss:0.32053
[8]	validation_0-logloss:0.29721
[9]	validation_0-logloss:0.27799
[10]	validation_0-logloss:0.26030
[11]	validation_0-logloss:0.24604
[12]	validation_0-logloss:0.23156
[13]	validation_0-logloss:0.22005
[14]	validation_0-logloss:0.20857
[15]	validation_0-logloss:0.19999
[16]	validation_0-logloss:0.19012
[17]	validation_0-logloss:0.18182
[18]	validation_0-logloss:0.17473
[19]	validation_0-logloss:0.16766
[20]	validation_0-logloss:0.15820
[21]	validation_0-logloss:0.15473
[22]	validation_0-logloss:0.14895
[23]	validation_0-logloss:0.14331
[24]	validation_0-logloss:0.13634
[25]	validation_0-logloss:0.13278
[26]	validation_0-logloss:0.12791
[27]	validation_0-logloss:0.12526
[28]	validation_0-logloss:0.11998
[29]	validation_0-loglos

In [12]:
xbg = XGBClassifier(n_estimators=400, learning_rate=0.01, max_depth=3)

evals = [(X_test, y_test)]

xbg.fit(X_train, y_train, early_stopping_rounds=70, eval_set=evals, eval_metric='logloss')
x_preds= xbg.predict(X_test)

[0]	validation_0-logloss:0.68479
[1]	validation_0-logloss:0.67657
[2]	validation_0-logloss:0.66854
[3]	validation_0-logloss:0.66066
[4]	validation_0-logloss:0.65294
[5]	validation_0-logloss:0.64537
[6]	validation_0-logloss:0.63794
[7]	validation_0-logloss:0.63065
[8]	validation_0-logloss:0.62350
[9]	validation_0-logloss:0.61643
[10]	validation_0-logloss:0.60953
[11]	validation_0-logloss:0.60277
[12]	validation_0-logloss:0.59613
[13]	validation_0-logloss:0.58960
[14]	validation_0-logloss:0.58320
[15]	validation_0-logloss:0.57691
[16]	validation_0-logloss:0.57067
[17]	validation_0-logloss:0.56460
[18]	validation_0-logloss:0.55863
[19]	validation_0-logloss:0.55284
[20]	validation_0-logloss:0.54708
[21]	validation_0-logloss:0.54149
[22]	validation_0-logloss:0.53602
[23]	validation_0-logloss:0.53054
[24]	validation_0-logloss:0.52532
[25]	validation_0-logloss:0.51994
[26]	validation_0-logloss:0.51463
[27]	validation_0-logloss:0.50959
[28]	validation_0-logloss:0.50454
[29]	validation_0-loglos

In [14]:
x_preds

array([1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [13]:
xbg = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)

evals = [(X_test, y_test)]

# 80번 학습을 필수 보장, 80번 이후엔 n_estimators에 결정트리 값을 400으로 설정해 놓았다 하더라도 400번 진행이 아닌
# loss 즉 손실값 확인 하면서 더이상 학습 의미가 없다 판단이 되면 중지
# 단, 코드는 동일하나 소소하게 다르게 결과 도출하기도 함 
xbg.fit(X_train, y_train, early_stopping_rounds=80, eval_set=evals, eval_metric='logloss')
x_preds= xbg.predict(X_test)

[0]	validation_0-logloss:0.61352
[1]	validation_0-logloss:0.54784
[2]	validation_0-logloss:0.49425
[3]	validation_0-logloss:0.44799
[4]	validation_0-logloss:0.40911
[5]	validation_0-logloss:0.37498
[6]	validation_0-logloss:0.34571
[7]	validation_0-logloss:0.32053
[8]	validation_0-logloss:0.29721
[9]	validation_0-logloss:0.27799
[10]	validation_0-logloss:0.26030
[11]	validation_0-logloss:0.24604
[12]	validation_0-logloss:0.23156
[13]	validation_0-logloss:0.22005
[14]	validation_0-logloss:0.20857
[15]	validation_0-logloss:0.19999
[16]	validation_0-logloss:0.19012
[17]	validation_0-logloss:0.18182
[18]	validation_0-logloss:0.17473
[19]	validation_0-logloss:0.16766
[20]	validation_0-logloss:0.15820
[21]	validation_0-logloss:0.15473
[22]	validation_0-logloss:0.14895
[23]	validation_0-logloss:0.14331
[24]	validation_0-logloss:0.13634
[25]	validation_0-logloss:0.13278
[26]	validation_0-logloss:0.12791
[27]	validation_0-logloss:0.12526
[28]	validation_0-logloss:0.11998
[29]	validation_0-loglos

In [ ]:
x_preds